In [1]:
!pip install CFEDemands --upgrade
!pip install xarray
!pip install dvc
!pip install --upgrade oauth2client 

Requirement already up-to-date: CFEDemands in /opt/conda/lib/python3.8/site-packages (0.2.7)


Requirement already up-to-date: oauth2client in /opt/conda/lib/python3.8/site-packages (4.1.3)


### (A) Estimate Demand System

In [2]:
#Nigeria consumption (own production) dataset

Nigeria_Data = '17L5cDhXRLNAckP3JvBLTLSYIguFqP2ebMvQLH96c0n4'

Retrieve data from google sheets using private keys:

In [3]:
import pandas as pd
import numpy as np
from eep153_tools import read_sheets

#### Need private keys from json file (we're authenticating using "service accounts")
!gpg --batch --passphrase "noodle octopus" -d ./students-9093fa174318.json.gpg > ./students-9093fa174318.json
####

# Add credentials if sheet not meant to be public
x = read_sheets(Nigeria_Data,sheet='Expenditures',json_creds='./students-9093fa174318.json')
                 
# Change 'ICRISAT' to key of your own sheet in Sheets, above
z = read_sheets(Nigeria_Data,sheet="HH Characteristics",json_creds='./students-9093fa174318.json')

# # Assume a single market: (Comment this out to make each village its own market)
x.insert(loc=2, column='m', value=1)
z.insert(loc=2, column='m', value=1)

gpg: AES256 encrypted data
gpg: encrypted with 1 passphrase


In [4]:
x = x.replace(0,np.nan) # Replace zeros with missing

#convert all values to float64
for i in range(3, len(x.columns)):
    x.iloc[:, i:] = x.iloc[:, i:].astype('float64')

In [5]:
x.set_index(['j', 't', 'm'])
z.set_index(['j','t','m'],inplace=True)

# Take logs of expenditures; call this y
y = np.log(x.set_index(['j', 't', 'm']))

This gives a dataframe of household characteristics $z$ with a simple structure.



In [6]:
x.head()

,t,j,m,"(Cocoyam, Spinach, etc)",Agricultural eggs,Animal fat,Apples,Avocado pear,Baby milk powder,Bananas,...,Sweet Potatoes,Tea,Tomato puree(canned),Tomatoes,Watermelon,Wheat flour,White beans,Wild game meat,Yam flour,Yam-roots
0,2010,10001,1,NaN,280.0,NaN,NaN,NaN,NaN,200.0,...,150.0,NaN,150.0,150.0,NaN,NaN,600.0,NaN,NaN,1500.0
1,2010,10002,1,NaN,280.0,NaN,NaN,NaN,NaN,180.0,...,200.0,140.0,240.0,120.0,NaN,NaN,400.0,NaN,NaN,1200.0
2,2010,10003,1,NaN,180.0,NaN,NaN,NaN,NaN,100.0,...,200.0,60.0,90.0,100.0,NaN,NaN,100.0,NaN,NaN,400.0
3,2010,10004,1,NaN,180.0,NaN,NaN,NaN,NaN,100.0,...,NaN,30.0,60.0,100.0,NaN,NaN,100.0,NaN,NaN,400.0
4,2010,10006,1,NaN,NaN,NaN,NaN,NaN,NaN,300.0,...,NaN,650.0,NaN,400.0,NaN,NaN,NaN,NaN,NaN,400.0


In [7]:
z.head()

,,,M 0-3,M 4-8,M 9-13,M 14-18,M 19-30,M 31-50,M 51+,F 0-3,F 4-8,F 9-13,F 14-18,F 19-30,F 31-50,F 51+
j,t,m,,,,,,,,,,,,,,
10001,2010,1,0,0,0,0,1,2,0,1,0,0,0,1,2,0
10002,2010,1,0,0,1,1,1,1,0,0,0,0,0,2,1,0
10003,2010,1,0,1,1,1,0,0,1,0,0,1,0,0,1,0
10004,2010,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1
10006,2010,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0


Also a dataframe of log expenditures $y$ with a similarly simple
structure.  Note that any expenditures of zero are dropped and
replaced with a missing value indicator.



In [8]:
y.head()

,,,"(Cocoyam, Spinach, etc)",Agricultural eggs,Animal fat,Apples,Avocado pear,Baby milk powder,Bananas,Beef,Beer (local and imported),Biscuits,...,Sweet Potatoes,Tea,Tomato puree(canned),Tomatoes,Watermelon,Wheat flour,White beans,Wild game meat,Yam flour,Yam-roots
j,t,m,,,,,,,,,,,,,,,,,,,,,
10001,2010,1,NaN,5.634790,NaN,NaN,NaN,NaN,5.298317,6.214608,6.291569,NaN,...,5.010635,NaN,5.010635,5.010635,NaN,NaN,6.396930,NaN,NaN,7.313220
10002,2010,1,NaN,5.634790,NaN,NaN,NaN,NaN,5.192957,7.090077,7.600902,NaN,...,5.298317,4.941642,5.480639,4.787492,NaN,NaN,5.991465,NaN,NaN,7.090077
10003,2010,1,NaN,5.192957,NaN,NaN,NaN,NaN,4.605170,6.214608,NaN,NaN,...,5.298317,4.094345,4.499810,4.605170,NaN,NaN,4.605170,NaN,NaN,5.991465
10004,2010,1,NaN,5.192957,NaN,NaN,NaN,NaN,4.605170,6.214608,NaN,NaN,...,NaN,3.401197,4.094345,4.605170,NaN,NaN,4.605170,NaN,NaN,5.991465
10006,2010,1,NaN,NaN,NaN,NaN,NaN,NaN,5.703782,5.703782,NaN,NaN,...,NaN,6.476972,NaN,5.991465,NaN,NaN,NaN,NaN,NaN,5.991465


## Estimation



With nothing more than this, we can estimate the demand system.  This
happens in two steps.  The first is the &ldquo;reduced form&rdquo; step:



In [9]:
import cfe

result = cfe.Result(y=y,z=z)

This creates a complicated &ldquo;Result&rdquo; object, with lots of different
attributes.  Note from below that attributes $y$ and $z$ are now defined.



In [10]:
result

<xarray.Result>
Dimensions:           (i: 43, j: 8522, k: 14, m: 1, t: 4)
Coordinates:
  * j                 (j) int64 10001 10002 10003 10004 ... 379151 379154 379155
  * m                 (m) int64 1
  * t                 (t) int64 2010 2012 2015 2018
  * i                 (i) <U36 '(Cocoyam, Spinach, etc)' ... 'Yam-roots'
  * k                 (k) <U7 'M 0-3' 'M 4-8' 'M 9-13' ... 'F 31-50' 'F 51+'
Data variables:
    alpha             object None
    beta              object None
    delta             object None
    prices            object None
    characteristics   (k, j, t, m) float64 0.0 1.0 1.0 1.0 ... nan nan nan 0.0
    loglambdas        object None
    a                 object None
    yhat              object None
    ce                object None
    cehat             object None
    e                 object None
    delta_covariance  object None
    beta_covariance   object None
    se_delta          object None
    se_beta           object None
    se_alpha          object None
    se_a              object None
    y                 (i, j, m, t) float64 nan -inf -inf -inf ... nan nan 7.824
    logp              object None
    z                 (k, j, t, m) float64 0.0 1.0 1.0 1.0 ... nan nan nan 0.0
Attributes:
    firstround:            2010
    min_proportion_items:  0.125
    min_xproducts:         30
    useless_expenditures:  False
    stderr_tol:            0.01
    indices:               Indices(j='j', t='t', m='m', i='i', k='k')
    verbose:               False

### First step of Estimation



Recall that there are two steps to estimation; the first step
involves estimating the &ldquo;reduced form&rdquo; linear regression 
$$
y_{it}^j = {a}_{it} + \delta_i'{z}^j_t + \epsilon_{it}^j.
$$

The Result class has code to estimate this in one line:



In [11]:
result.get_reduced_form()

AssertionError: No variation in non-missing data for item (Cocoyam, Spinach, etc).

After running this we can examine the estimated coefficients $\delta$:



In [ ]:
result.delta.to_dataframe().unstack('k')

Also the good-time constants $a_{it}$ (this captures the effects of prices)



In [ ]:
result.a.to_dataframe().unstack('i')

### Second step of Estimation



The second step involves using Singular Value Decomposition to find
the rank one matrix that best approximates the residuals $e_{it}^j$.
This can be interpreted as
$$
    -\beta_i\log\lambda^j_t,
$$
where the $\log\lambda^j_t$ is the log of the marginal utility of
expenditures (MUE) for household $j$ at time $t$, and where $\beta_i$ are
the corresponding &ldquo;Frisch elasticities&rdquo; that tell us how much
demand changes as the MUE falls.

Estimates can also be computed as a one-liner:



In [ ]:
result.get_beta(as_df=True)

That&rsquo;s all there is to estimation!  Note that we didn&rsquo;t estimate
demands for all goods&#x2014;lots of goods didn&rsquo;t have enough observations,
and were automatically dropped.  (This can be controlled using the
`min_proportion_items` and `min_xproducts` attributes when one
instantiates the result object.)



### Assessment of Fit



Now, let&rsquo;s see how we did, by comparing total expenditures predicted by the
model we&rsquo;ve estimated with actual total expenditures:



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm

xbar = np.exp(result.y).sum(['m','i']).to_dataframe('xbar').replace(0,np.nan).squeeze()
xhat = result.get_predicted_expenditures().sum(['m','i']).to_dataframe('xhat').replace(0,np.nan).squeeze()

# Make dataframe of actual & predicted
df = pd.DataFrame({'Actual':np.log(xbar),'Predicted':np.log(xhat)})

df.plot.scatter(x='Predicted',y='Actual')

# Add 45 degree line
v = plt.axis()
vmin = np.max([v[0],v[2]])
vmax = np.max([v[1],v[3]])
plt.plot([vmin,vmax],[vmin,vmax])

### Saving Result



You can save the estimated result, using something like



In [ ]:
result.to_dataset('icrisat.ds')

### (B) Nutritional content of different foods

For all the foods you're considering you'll need to be able to describe their nutritional content, in terms that allow you to compare with recommended daily allowances.

### (B) Engel's Law

Explore the validity of Engel's Law in the context of our population

### (B) Nutritional adequacy of diet

Given the food actually consumed in your data, what can you say about the adequacy of the diets in the population you're studying? What proportion of households consume enough so that members will exceed dietary recommendations? What proportion do not?

### (C) Counterfactual Experiments

If everyone's total food expenditures doubled (holding prices fixed) how would this affect nutrition in the population? What if relative food prices changed? Think of other counterfactual experiments that might be interesting.